#**IMPLEMENTACIÓN Y PRUEBA DEL ALGORITMO APRIORI**

##Estudiante: Charlie Joel Luna Ccasani
##Código: 161368

##Importamos librerías

In [1]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  

##Leemos el dataset

In [5]:
Data = np.load('spotify.npy',allow_pickle=True)
print("¡Se han registrado ",len(Data.tolist())," play list!")

¡Se han registrado  10000  play list!


##Realizamos un preprocesamiento de datos (ordenar alfabéticamente y todo a mayúsculas)

In [37]:
#Utlizaremos la primera décima parte de los datos
#Recuperaremos los n primeros datos del .npy en una matriz
def Convertir_a_Mayusculas(Lista):
  for K in range(len(Lista)):
    Lista[K]=Lista[K].upper()
  return(Lista)
def ExtraerDatos(Data,N):
  SubData=[]
  for K in range(N):
    Aux=Data.tolist()[K]
    Aux.sort()
    Aux=Convertir_a_Mayusculas(Aux)
    SubData=SubData+[Aux]
  print("Extraímos ",len(SubData)," listas de ",len(Data.tolist()))
  return(SubData)
#Utlizamos el módulo creado
#Extraemos 1000 play list
Datos=ExtraerDatos(Data,200)

Extraímos  200  listas de  10000


##Algoritmo Apriori

In [38]:
#Módulo que obtiene el supor count de un data set
#Input : Matriz (lista de listas)
#Output: Diccionario de elementos diferentes y suport
def ContarItemsDiferentes(Data):
  ItemSet={}
  for Fila in Data:
    for Item in Fila:
      if Item not in ItemSet:
        ItemSet[Item]=1
      else:
        ContadorAux=ItemSet.get(Item)
        ItemSet.update({Item:ContadorAux+1})
  return ItemSet
#----------------------------
Item=ContarItemsDiferentes(Datos)
print("Se registraron ",len(Item)," canciones diferentes")

Se registraron  9036  canciones diferentes


In [39]:
#Módulo para ver algunas canciones encontradas
def MostrarAlgunasCanciones(Diccionario,N):
  K=0
  for key,values in Diccionario.items():
    if(K<N):
      print(key," - ",values)
    else:
      break
    K=K+1
#-------------------------
MostrarAlgunasCanciones(Item,10)

A THOUSAND MILES  -  4
ALL THE SMALL THINGS  -  5
BABY  -  4
BEAUTIFUL SOUL  -  3
BEEP  -  1
BUTTONS  -  3
CHECK ON IT - FEAT. BUN B AND SLIM THUG  -  1
CHECK YES JULIET  -  1
CLOSER  -  18
CRAZY IN LOVE  -  7


##Definimos algunos módulos que ayudarán para realizar el algoritmo Apriori

In [40]:
#Módulo para buscar una SubLista en una Lista
def BuscarSubItems1(SubLista,Lista):
    Cont=0
    for Item in SubLista:
        if(Item in Lista):
            Cont+=1
        else:
            break
    if(Cont==len(SubLista)):
        return True
    else:
        return False

#Módulo para buscar una SubLista en una matriz de Listas
def BuscarSubItems2(SubItem,Lista):
    Contador=0
    for SubLista in Lista:
        if(BuscarSubItems1(SubItem,SubLista)==True):
            Contador+=1
    return(Contador)

#Combinar elementos de una lista
def CombinarTodos(Lista):
    Combinaciones=[]
    Inicio=1
    Fin=len(Lista)
    for Item in Lista:
        for K in range(Inicio,Fin):
            Aux=[Item,Lista[K]]
            Combinaciones=Combinaciones+[Aux]
        Inicio+=1
    print("Se registró ",len(Combinaciones)," combinaciones")
    return Combinaciones

#Generar una lista desde un diccionario analizando el MinSuport
def GenerarListaDeUnDiccionario(Dic,MinSuport):
    Lista=[]
    for Key,Values in Dic.items():
        if(Values>=MinSuport):
            Lista+=[Key]
    return Lista

#Recuperar combinaciones que superan el minsuport
def FiltrarCombinaciones(Combinaciones,Datos,MinSuport):
    Suport=[]
    IndicesParaEliminar=[]
    for K in range(len(Combinaciones)):
        X=BuscarSubItems2(Combinaciones[K],Datos)
        if(X<MinSuport):
            IndicesParaEliminar=IndicesParaEliminar+[K]
        Suport=Suport+[X]
    print("Se han eliminado ",len(IndicesParaEliminar)," combinaciones")
    Fin=len(IndicesParaEliminar)-1
    for J in range(Fin,-1,-1):
        Combinaciones.pop(IndicesParaEliminar[J])
        Suport.pop(IndicesParaEliminar[J])
    print("Tenemos ",len(Combinaciones)," combinaciones")
    return(Combinaciones,Suport)

##Aplicamos todas las funciones previamente definidas para obtener las relaciones más relevantes

In [41]:
# 1.Generar matriz de los datos
# 2.Contar elementos diferentes y la frecuencia de estos en un Diccionario
# 3.Generar la lista base para generar la primera combinación
MinSuport=5
Lista=GenerarListaDeUnDiccionario(Item,MinSuport)
# 4.Primera combinación
Combinaciones=CombinarTodos(Lista)
# 5.Filtrar las combinaciones que superan al MinSuport
Combinaciones,Soporte=FiltrarCombinaciones(Combinaciones,Datos,MinSuport)
print("Se han registrado ",len(Combinaciones)," que superaron el minsuport = ",MinSuport)

Se registró  30628  combinaciones
Se han eliminado  30601  combinaciones
Tenemos  27  combinaciones
Se han registrado  27  que superaron el minsuport =  5


##Definimos un módulo para calcular la confianza de la regla encontrada

In [44]:
def Confianza(ITEM,Regla,Suport):
  X=Regla[0]
  Denominador=ITEM[X]
  return(Suport/Denominador)

##Mostramos algunas reglas que superan la confianza del 60% y un minsuport de 5

In [54]:
#Mostrando resultados
def GenerarReglas(ITEMS,ItemsFrecuentes,SoporteDeItems,Confidence):
  K=0
  Top=0
  for K in range(len(ItemsFrecuentes)):
    Confi=Confianza(ITEMS,ItemsFrecuentes[K],SoporteDeItems[K])
    if(Confi>Confidence):
      Text=ItemsFrecuentes[K]
      Text1=Text[0]
      Text2=Text[1]
      Top+=1
      print("________________ N°",Top,"______________________")
      print("    Regla :",Text1+" -> ",Text2)
      print("  Soporte :",SoporteDeItems[K])
      print("Confianza :",round((Confi*100),2),"%")
      print("")
GenerarReglas(Item,Combinaciones,Soporte,0.6)

________________ N° 1 ______________________
    Regla : PARTY IN THE U.S.A. ->  SORRY
  Soporte : 5
Confianza : 71.43 %

________________ N° 2 ______________________
    Regla : GO FLEX ->  GOOSEBUMPS
  Soporte : 5
Confianza : 100.0 %

________________ N° 3 ______________________
    Regla : CAKE BY THE OCEAN ->  SORRY
  Soporte : 5
Confianza : 83.33 %

________________ N° 4 ______________________
    Regla : SHUT UP AND DANCE ->  SORRY
  Soporte : 5
Confianza : 83.33 %

________________ N° 5 ______________________
    Regla : BROCCOLI (FEAT. LIL YACHTY) ->  HUMBLE.
  Soporte : 7
Confianza : 63.64 %

________________ N° 6 ______________________
    Regla : CAN'T STOP THE FEELING! (ORIGINAL SONG FROM DREAMWORKS ANIMATION'S "TROLLS") ->  WORK FROM HOME
  Soporte : 5
Confianza : 100.0 %

________________ N° 7 ______________________
    Regla : ON MY MIND ->  ROSES
  Soporte : 5
Confianza : 100.0 %

________________ N° 8 ______________________
    Regla : ON MY MIND ->  SORRY
  Soporte : 

###NOTA: El resultado muestra las mejores 12 reglas obtenidas; se consideró un minsuport de 5 en vista de que utilizamos pocos datos (200); sin embargo, filtramos las reglas con una confianza del 60% para obtener resultados de calidad.

###NOTA 2: Analizaremos las siguientes reglas con 100% de confianza y soporte 5
1. GO FLEX ->  GOOSEBUMPS, ambas canciones son de Hip-hop/rap
2. CAN'T STOP THE FEELING! (ORIGINAL SONG FROM DREAMWORKS ANIMATION'S "TROLLS") ->  WORK FROM HOME, ambas canciones son Pop
3. ON MY MIND ->  ROSES, en este caso el género de las canciones son diferentes pero no son tan lejanas (Pop y Rock)
4. ON MY MIND ->  SORRY, ambas canciones son Pop